Showing gene expression changes along the crypt villus axis

In [ ]:
import scanpy as sc
import numpy as np
from typing import List
import scvelo as scv
import matplotlib.colors as clr
import matplotlib.pyplot as plt

In [2]:
# slightly modified to make it less red
zissou = [
    "#3A9AB2",
    "#6FB2C1",
    "#91BAB6",
    "#A5C2A3",
    "#BDC881",
    "#DCCB4E",
    "#E3B710",
    "#EC7A05",
    "#F11B00",
]

colormap = clr.LinearSegmentedColormap.from_list("Zissou", zissou)
colormap_r = clr.LinearSegmentedColormap.from_list("Zissou", zissou[::-1])

load in adata

In [3]:
adata = sc.read_h5ad('uninfected/analysis/cleaned/final_object_no_peyers.h5ad')

In [ ]:
# Normalize
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata

In [5]:
def filter_adata_expressed_in_n_cells(adata, percent=0.05):
    """
    Filter the adata to only include genes expressed in more than a certain percentage of cells

    Parameters:
    - adata (anndata): The anndata object containing the gene expression
    - percent (float): The percentage of cells a gene must be expressed in to be included

    Return:
    - adata (anndata): The filtered anndata object
    """
    bin_Layer = adata.X > 0
    gene_expressed_in_percent_cells = np.mean(bin_Layer, axis=0)
    keep = gene_expressed_in_percent_cells > percent
    adata = adata[:, keep]
    return adata

In [6]:
def scvelo_heatmap(
    adata: sc.AnnData,
    batches: List[str],
    sortby: str,
    highlight: List[str],
    n_bins: int = 5,
    dpi = 350
):

    print("Creating Heatmap for batches", " + ".join(batches))
    # Subset batches
    adata = adata[adata.obs["batch"].isin(batches)]
    # Filter to include only genes that are expressed in 5% of the cells
    #adata = filter_adata_expressed_in_n_cells(adata)
    adata = adata.copy()

    n_convolve = len(adata) // n_bins
    print(f"Setting `n_convolve` to {n_convolve} ({n_bins} bins, {len(adata)} cells) ")
    # Plot
    s = scv.pl.heatmap(
        adata,
        var_names=adata.var_names,
        sortby=sortby,
        n_convolve=n_convolve,
        show=False,
        yticklabels=True,
        rasterized=True,
        color_map=colormap,
    )
    ax = s.ax_heatmap
    
    highlight_lower = [h.lower() for h in highlight]
   # Loop through the y-axis tick labels and show/hide based on the 'highlight' list
    highlighted_genes = []
    for i, label in enumerate(ax.get_yticklabels()):
        gene_name = label.get_text().strip()  # Remove any leading/trailing whitespace
        if gene_name.lower() in highlight_lower:
            highlighted_genes.append(gene_name)
            label.set_visible(True)
            ax.get_yticklines()[2 * i + 1].set_visible(True)
        else:
            label.set_visible(False)
            ax.get_yticklines()[2 * i + 1].set_visible(False)
        ax.get_yticklines()[2 * i].set_visible(False)

    # Print information about highlighted genes
    print(f"Highlighted genes: {len(highlighted_genes)} out of {len(highlight)}")
    print("Genes not found in the plot:", set(highlight) - set(highlighted_genes))

    ax.set_xlabel("")
    ax.set_title(f"Gene Expression Trends Along {sortby}")

    plt.savefig('../figures/cv_axis_xytokines.pdf', dpi=dpi, bbox_inches='tight')
    return s

In [7]:
highlight = [
    
    "Il18",
    "Tgfb1",
    "Tgfb2",
    "Tgfb3",
    "Cxcl9",
    "Cxcl10",
    "Cxcl16",
    "Itgae",
    "Il15",
]

batches = {
    "segmentation_SI1": {"x": 6200, "y": 6200},
    "segmentation_SI2": {"x": 5800, "y": 5500},
}

In [ ]:
s = scvelo_heatmap(
    adata[::10, :],
    batches=list(batches.keys()),
    sortby="crypt_villi_axis",
    highlight=highlight,
    n_bins=10,
)